In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark import SQLContext
from itertools import islice
from pyspark.sql.functions import col

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
26,application_1596057829763_0027,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Load Data
[reference](https://stackoverflow.com/questions/53567185/reading-csv-from-pyspark-specifying-schema-wrong-types)

In [2]:
# Below syntax can load csv into dataframe with a proper data type.
df = spark.read.option('header','true').option("inferSchema","true").csv('s3://dataset-project/US_Accidents_state_clean_data.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Source: string (nullable = true)
 |-- TMC: double (nullable = true)
 |-- Severity: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Side: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- Visibility(mi): double (nullable = true)
 |-- Wind_Direction: string (nullable = true)
 |-- Weather_Condition: string (nullable = true)
 |-- Amenity: double (nullable = true)
 |-- Bump: double (nullable = true)
 |-- Crossing: double (nullable = true)
 |-- Give_Way: double (nullable = true)
 |-- Junction: double (nullable = true)
 |-- No_Exit: double (nullable = true)
 |-- Railway: double (nullable = true)
 |-- Roundabout: double (nullable = true)
 |-- Station: dou

In [4]:
df.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('Source', 'string'), ('TMC', 'double'), ('Severity', 'double'), ('Start_Lng', 'double'), ('Start_Lat', 'double'), ('Distance(mi)', 'double'), ('Side', 'string'), ('City', 'string'), ('County', 'string'), ('Timezone', 'string'), ('Temperature(F)', 'double'), ('Humidity(%)', 'double'), ('Pressure(in)', 'double'), ('Visibility(mi)', 'double'), ('Wind_Direction', 'string'), ('Weather_Condition', 'string'), ('Amenity', 'double'), ('Bump', 'double'), ('Crossing', 'double'), ('Give_Way', 'double'), ('Junction', 'double'), ('No_Exit', 'double'), ('Railway', 'double'), ('Roundabout', 'double'), ('Station', 'double'), ('Stop', 'double'), ('Traffic_Calming', 'double'), ('Traffic_Signal', 'double'), ('Turning_Loop', 'double'), ('Sunrise_Sunset', 'string'), ('Hour', 'double'), ('Weekday', 'string'), ('Time_Duration(min)', 'double')]

In [5]:
cols = df.columns
cols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Source', 'TMC', 'Severity', 'Start_Lng', 'Start_Lat', 'Distance(mi)', 'Side', 'City', 'County', 'Timezone', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Hour', 'Weekday', 'Time_Duration(min)']

## Data Preprocessing

#### Categorical Data

In [6]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
# remove 'Timezone' from the categoricalColumns list since only have one distinct value in the cols
categoricalColumns = ['Source', 'Side', 'City', 'County', 'Timezone', 'Wind_Direction', 'Weather_Condition', 'Sunrise_Sunset', 'Weekday']
stages = []

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Label Data

In [7]:
label_stringIdx = StringIndexer(inputCol = 'Severity', outputCol = 'label')
stages += [label_stringIdx]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Numeric Data

In [8]:
features_col = df.columns[:]
numericCols = set(features_col) - set(categoricalColumns) - set(['Severity'])
numericCols = list(numericCols)
numericCols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Visibility(mi)', 'Railway', 'Give_Way', 'Traffic_Calming', 'Humidity(%)', 'Amenity', 'No_Exit', 'Stop', 'TMC', 'Station', 'Start_Lng', 'Bump', 'Hour', 'Time_Duration(min)', 'Start_Lat', 'Roundabout', 'Junction', 'Crossing', 'Traffic_Signal', 'Pressure(in)', 'Distance(mi)', 'Turning_Loop', 'Temperature(F)']

In [9]:
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Build pipeline model

In [10]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=stages)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Fit pipeline model

In [11]:
pipeline_model = pipeline.fit(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Transform data

df = pipeline_model.transform(df)
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- Source: string (nullable = true)
 |-- TMC: double (nullable = true)
 |-- Severity: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Side: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- Visibility(mi): double (nullable = true)
 |-- Wind_Direction: string (nullable = true)
 |-- Weather_Condition: string (nullable = true)
 |-- Amenity: double (nullable = true)
 |-- Bump: double (nullable = true)
 |-- Crossing: double (nullable = true)
 |-- Give_Way: double (nullable = true)
 |-- Junction: double (nullable = true)
 |-- No_Exit: double (nullable = true)
 |-- Railway: double

In [13]:
(train, test) = df.randomSplit([0.8, 0.2], seed = 0)
print("Training Dataset Count: " + str(train.count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training Dataset Count: 219444

In [14]:
print("Test Dataset Count: " + str(test.count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Dataset Count: 54784

## Modeliing

### Random Forest

In [15]:
# Train a RandomForest model.
random_forest = RandomForestClassifier(featuresCol='features', labelCol='label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Running without crossvalidation

In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
rfModel = random_forest.fit(train)
predictions = rfModel.transform(test)
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Area Under ROC: 0.8183046489337149

#### Running with crossvalidation and gridsearch

In [17]:
from pyspark.ml.tuning import ParamGridBuilder

param_grid = ParamGridBuilder().\
    addGrid(random_forest.maxDepth, [2, 3, 4]).\
    addGrid(random_forest.minInfoGain, [0.0, 0.1, 0.2, 0.3]).\
    build()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
evaluator = BinaryClassificationEvaluator()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
from pyspark.ml.tuning import CrossValidator

crossvalidation = CrossValidator(estimator=random_forest, estimatorParamMaps=param_grid, evaluator=evaluator)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
crossvalidation_mod = crossvalidation.fit(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
pred_train = crossvalidation_mod.transform(train)
pred_train.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+--------+-----+--------+----------+------------------+------------+----+-------+------+----------+--------------+-----------+------------+--------------+--------------+-----------------+-------+----+--------+--------+--------+-------+-------+----------+-------+----+---------------+--------------+------------+--------------+----+-------+------------------+--------------------+--------------------+----------+
|label|            features|  Source|  TMC|Severity| Start_Lng|         Start_Lat|Distance(mi)|Side|   City|County|  Timezone|Temperature(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Weather_Condition|Amenity|Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station|Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|Sunrise_Sunset|Hour|Weekday|Time_Duration(min)|       rawPrediction|         probability|prediction|
+-----+--------------------+--------+-----+--------+----------+------------------+------------+----+-------+------+-------

In [22]:
pred_test = crossvalidation_mod.transform(test)
pred_test.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+--------+-----+--------+------------------+------------------+------------+----+-------+------+----------+--------------+-----------+------------+--------------+--------------+-----------------+-------+----+--------+--------+--------+-------+-------+----------+-------+----+---------------+--------------+------------+--------------+----+-------+------------------+--------------------+--------------------+----------+
|label|            features|  Source|  TMC|Severity|         Start_Lng|         Start_Lat|Distance(mi)|Side|   City|County|  Timezone|Temperature(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Weather_Condition|Amenity|Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station|Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|Sunrise_Sunset|Hour|Weekday|Time_Duration(min)|       rawPrediction|         probability|prediction|
+-----+--------------------+--------+-----+--------+------------------+------------------+------------+---

In [23]:
print('Random Forest Accuracy on training data (areaUnderROC): ', evaluator.setMetricName('areaUnderROC').evaluate(pred_train), "\n"
      'Random Forest Accuracy on test data (areaUnderROC): ', evaluator.setMetricName('areaUnderROC').evaluate(pred_test))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Random Forest Accuracy on training data (areaUnderROC):  0.8034820192347665 
Random Forest Accuracy on test data (areaUnderROC):  0.8038998852712421

### Logistic Regression

In [24]:
from pyspark.ml.classification import LogisticRegression
logr = LogisticRegression(featuresCol='features', labelCol='label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
lrModel = logr.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
predictions = lrModel.transform(test)
predictions.select('label', 'rawPrediction', 'prediction', 'probability').show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+----------+--------------------+
|label|       rawPrediction|prediction|         probability|
+-----+--------------------+----------+--------------------+
|  0.0|[5.5420819641506,...|       0.0|[0.99048081070368...|
|  0.0|[3.14927920982845...|       0.0|[0.50695105515947...|
|  0.0|[4.80927036296938...|       0.0|[0.95847496091541...|
|  0.0|[4.73885729609672...|       0.0|[0.95060408417238...|
|  0.0|[4.62976972229008...|       0.0|[0.94272657064931...|
|  0.0|[4.51386600936744...|       0.0|[0.93106923442645...|
|  0.0|[4.30806966698620...|       0.0|[0.90296495406164...|
|  0.0|[4.31352749215816...|       0.0|[0.90058958252897...|
|  0.0|[3.95427443999306...|       0.0|[0.82445811421304...|
|  0.0|[2.96766618775138...|       1.0|[0.43199008550508...|
+-----+--------------------+----------+--------------------+
only showing top 10 rows

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print('Logistic Regression Test Area Under ROC', evaluator.evaluate(predictions))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Logistic Regression Test Area Under ROC 0.8549035956490494

### References

[Tutorial](https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa)

[PySpark Guide](https://mingchen0919.github.io/learning-apache-spark/index.html)